In [30]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
%matplotlib inline
import numpy as np
import json


#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim

import optuna
from optuna.trial import Trial

import pickle


pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions
from multimodal.helper_functions import   reproducible_comp,  get_final_df
from multimodal.model_architecture import Generic1DCNN
reproducible_comp()

In [33]:
file_path = "../../../data/processed_data/force_inner.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_force_inner, category_force_inner, category_names, df_force_inner = get_final_df(data, is_acc_data=0, is_multilabel=1)

labels = category_force_inner.copy()
features=features_force_inner.copy()
print("Features Shape: ", features.shape)
print("Labels Shape: ", labels.shape)

Features Shape:  (1350, 1)
Labels Shape:  (1350, 4)


In [14]:
max_length = 150
for i in range (len(features)):
    if(len(features[i][0]<max_length)):
        features[i][0]=np.pad(features[i][0], (0,max_length-len(features[i][0])), 'constant', constant_values=(0))
feat= np.zeros(len(features) *max_length).reshape(-1, max_length)
for i in range(len(features)):
    feat[i]= np.concatenate((features[i][0]), axis=None)
    

print("Updated Features Shape: ", feat.shape)

Updated Features Shape:  (1350, 150)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(feat, labels, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels)

In [16]:
X_train = X_train.reshape(-1, 1, feat.shape[1])
X_test = X_test.reshape(-1, 1, feat.shape[1])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1215, 1, 150)
(1215, 4)
(135, 1, 150)
(135, 4)


In [23]:
from torch.utils.data import DataLoader, TensorDataset

# Convert your data to PyTorch tensors
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

input_channels = 1
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the objective function for hyperparameter optimization
def objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        hidden_channels = trial.suggest_int("hidden_channels", 16, 256)
        n_layers = trial.suggest_int("n_layers", 1, 7)
        kernel_size = trial.suggest_int("kernel_size", 3, 9)
        stride = trial.suggest_int("stride", 1, 7)
        padding = trial.suggest_int("padding", 1, 10)
        pooling_type = trial.suggest_categorical("pooling_type", ["max", "avg"])  
        use_batch_norm = trial.suggest_categorical("use_batch_norm", [True, False]) 


        # Split the data into training and validation sets
        x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(
            x_train, y_train, test_size=0.2, random_state=42
        )

        train_dataset = TensorDataset(x_train_split, y_train_split)
        valid_dataset = TensorDataset(x_valid_split, y_valid_split)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

        # Create the model with the suggested hyperparameters
        model = Generic1DCNN(input_channels, num_classes, n_layers, kernel_size, stride, padding,
                            hidden_channels, dropout_prob, pooling_type, use_batch_norm).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for inputs_valid, labels_valid in valid_loader:
                    inputs_valid, labels_valid = inputs_valid.to(device), labels_valid.to(device)
                    outputs_valid = model(inputs_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss
    


In [24]:
# Create an Optuna study with SQLite storage and TPE pruner
study = optuna.create_study(
    study_name="Case#1: Force Inner",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),  
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 100  

# Perform hyperparameter optimization
study.optimize(objective, n_trials=num_trials, n_jobs=1)


[I 2023-10-09 02:42:05,018] A new study created in RDB with name: Case#1: Force Inner
[I 2023-10-09 02:42:08,498] Trial 0 finished with value: 1.0 and parameters: {'lr': 0.004142172754578139, 'dropout_prob': 0.17682934106212006, 'hidden_channels': 19, 'n_layers': 7, 'kernel_size': 5, 'stride': 6, 'padding': 2, 'pooling_type': 'max', 'use_batch_norm': False}. Best is trial 0 with value: 1.0.


Trial failed with error: Given input size: (19x1x1). Calculated output size: (19x1x0). Output size is too small
Epoch 10/50, Training Loss: 0.627814307808876, Validation Loss: 0.6386155635118484
Epoch 20/50, Training Loss: 0.6139178797602654, Validation Loss: 0.6248702704906464
Epoch 30/50, Training Loss: 0.5190654117614031, Validation Loss: 0.5339046120643616
Epoch 40/50, Training Loss: 0.4045169521123171, Validation Loss: 0.39792145788669586


[I 2023-10-09 02:42:13,598] Trial 1 finished with value: 0.3257243186235428 and parameters: {'lr': 0.0001697554862615658, 'dropout_prob': 0.13453233345703702, 'hidden_channels': 104, 'n_layers': 6, 'kernel_size': 3, 'stride': 4, 'padding': 9, 'pooling_type': 'avg', 'use_batch_norm': False}. Best is trial 1 with value: 0.3257243186235428.


Epoch 50/50, Training Loss: 0.3414702098816633, Validation Loss: 0.3257243186235428
Epoch 10/50, Training Loss: 0.09028405556455255, Validation Loss: 0.062729905359447
Epoch 20/50, Training Loss: 0.02905191230820492, Validation Loss: 0.040175887290388346
Epoch 30/50, Training Loss: 0.01925289141945541, Validation Loss: 0.045288830529898405
Epoch 40/50, Training Loss: 0.016525819766684435, Validation Loss: 0.03841713350266218


[I 2023-10-09 02:42:18,437] Trial 2 finished with value: 0.04237414291128516 and parameters: {'lr': 0.0013525404648470315, 'dropout_prob': 0.24743426607456803, 'hidden_channels': 219, 'n_layers': 5, 'kernel_size': 5, 'stride': 6, 'padding': 6, 'pooling_type': 'max', 'use_batch_norm': False}. Best is trial 2 with value: 0.04237414291128516.


...

Epoch 50/50, Training Loss: 2.606295021223559e-05, Validation Loss: 0.0005970457841613097
Epoch 10/50, Training Loss: 0.005949929234702722, Validation Loss: 0.007754370337352157
Epoch 20/50, Training Loss: 0.0006012226681377797, Validation Loss: 0.0028713717401842587
Epoch 30/50, Training Loss: 0.00023469786526675307, Validation Loss: 0.0014927695447113365
Epoch 40/50, Training Loss: 9.4792637298724e-05, Validation Loss: 0.0013087699026073096


[I 2023-10-09 02:48:34,169] Trial 98 finished with value: 0.0014532084014717839 and parameters: {'lr': 0.006801335123023432, 'dropout_prob': 0.05814389202094134, 'hidden_channels': 184, 'n_layers': 2, 'kernel_size': 6, 'stride': 3, 'padding': 3, 'pooling_type': 'max', 'use_batch_norm': True}. Best is trial 81 with value: 6.592806153093989e-05.


Epoch 50/50, Training Loss: 5.856736163423193e-05, Validation Loss: 0.0014532084014717839
Epoch 10/50, Training Loss: 0.005847896709383349, Validation Loss: 0.017683283789665438
Epoch 20/50, Training Loss: 0.0005559271621677908, Validation Loss: 0.004284071970687364
Epoch 30/50, Training Loss: 7.393779787889798e-05, Validation Loss: 0.0006425645951821934
Epoch 40/50, Training Loss: 3.6342027499358664e-05, Validation Loss: 0.000499819445394678


[I 2023-10-09 02:48:38,633] Trial 99 finished with value: 0.00043938882072325214 and parameters: {'lr': 0.003059283758420686, 'dropout_prob': 0.010175605718229556, 'hidden_channels': 233, 'n_layers': 3, 'kernel_size': 7, 'stride': 2, 'padding': 2, 'pooling_type': 'max', 'use_batch_norm': True}. Best is trial 81 with value: 6.592806153093989e-05.


Epoch 50/50, Training Loss: 2.3368625420516764e-05, Validation Loss: 0.00043938882072325214


In [25]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 0.006170767628576648, 'dropout_prob': 0.03377115813672493, 'hidden_channels': 73, 'n_layers': 4, 'kernel_size': 7, 'stride': 2, 'padding': 4, 'pooling_type': 'max', 'use_batch_norm': True}
Best final validation loss:
6.592806153093989e-05


In [32]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/force_inner_best_configs.json"

# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [31]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You